In [ ]:
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import tf_clahe
import os
from tqdm import tqdm
from glob import glob

IMG_H = 271
IMG_W = 481
IMG_C = 3  ## Change this to 1 for grayscale.

In [ ]:
'''
    Function for Balance Contrast Enhancement Technique (BCET)
    This technique provides solution to biased color (RGB) composition. 
    The contrast of the image can be stretched or compressed without changing the histogram pattern of the input image(x).
    The solution is based on the parabolic function obtained from the input image.
'''
@tf.function
def bcet(img):

    
    Lmin = tf.reduce_min(img) # MINIMUM OF INPUT IMAGE
#     Lmin = np.min(img) # MINIMUM OF INPUT IMAGE
#     print("Lmin", Lmin)
    Lmax = tf.reduce_max(img) # MAXIMUM OF INPUT IMAGE
#     Lmax = np.max(img) # MAXIMUM OF INPUT IMAGE
#     print("Lmax", Lmax)
    Lmean = tf.reduce_mean(img) #MEAN OF INPUT IMAGE
#     Lmean = np.mean(img) #MEAN OF INPUT IMAGE
#     print("Lmean", Lmean)
    LMssum = tf.reduce_mean(img * img) #MEAN SQUARE SUM OF INPUT IMAGE
#     LMssum = np.mean(pow(img, 2)) #MEAN SQUARE SUM OF INPUT IMAGE
#     print("LMssum", LMssum)

    Gmin = tf.constant(0, dtype="float32") #MINIMUM OF OUTPUT IMAGE
    Gmax = tf.constant(255, dtype="float32") #MAXIMUM OF OUTPUT IMAGE
    Gmean = tf.constant(110, dtype="float32") #MEAN OF OUTPUT IMAGE
    
    subber = tf.constant(2, dtype="float32")
    
    # find b
    
    bnum = ((Lmax**subber)*(Gmean-Gmin)) - (LMssum*(Gmax-Gmin)) + ((Lmin**subber) *(Gmax-Gmean))
    bden = subber * ((Lmax*(Gmean-Gmin)) - (Lmean*(Gmax-Gmin)) + (Lmin*(Gmax-Gmean)))
    
    b = bnum/bden
    
    # find a
    a1 = Gmax-Gmin
    a2 = Lmax-Lmin
    a3 = Lmax+Lmin-(subber*b)
            
    a = a1/(a2*a3)
    
    # find c
    c = Gmin - (a*(Lmin-b)**subber)
    
    # Process raster
    y = a*((img - b)**subber) + c #PARABOLIC FUNCTION

    return y

def bcet_processing(img,channels=3):
    img = tf.cast(img, tf.float32)
    layers = [None] * 3
    for i in range(channels):
        layer = img[:,:,i]
        layer = bcet(layer)
        layers[i] = layer
       
        
    final_image = tf.stack(layers, axis=-1)

    return final_image

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def prepare(x):
    

    ### implement BCET to iamges
    # x = bcet_processing(x)
    
    ### implement clahe
    # x = tf_clahe.clahe(x)
    
    ### Convert RGB to CIE XYZ
    
    # x = tfio.experimental.color.rgb_to_xyz(x)
    # # # convert back uint8
    
    
    # x = tfio.experimental.color.rgb_to_ycbcr(x)
    # x = tf.cast(x, tf.float32) / 255.0
    
    # x = tfio.experimental.color.rgb_to_bgr(x)
    x = tf.image.adjust_contrast(x, 9.)
    x = tf.image.adjust_hue(x, 9.)
    x = tf.image.adjust_gamma(x)
    x = tfa.image.median_filter2d(x)
    # x = tf.cast(x * 255.0, tf.uint8)
    
    # x = tfa.image.equalize(x)
    return x

In [ ]:
def read_data_with_labels(filepath, class_names):
    image_list = []
    label_list = []
    for class_n in class_names:  # do dogs and cats
        path = os.path.join(filepath,class_n)  # create path to dogs and cats
        class_num = class_names.index(class_n)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  
            if ".DS_Store" != img:
                filpath = os.path.join(path,img)
#                 print(filpath, class_num)
                image_list.append(filpath)
                label_list.append(class_num)
#     print(image_list, label_list)
    return image_list, label_list

def load_image_with_label(image_path, label):
    class_names = ["normal", "defect"]
#     print(image_path)
    img = tf.io.read_file(image_path)
    img = tf.io.decode_bmp(img, channels=IMG_C)
    img = prepare(img)
    img = tf.cast(img, tf.float32)
    #     rescailing image from 0,255 to -1,1
    # img = (img - 127.5) / 127.5
    
    return img, label

def tf_dataset_labels(images_path, batch_size, class_names=None):
    
    filenames, labels = read_data_with_labels(images_path, class_names)
#     print("testing")
#     print(filenames, labels)
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.shuffle(buffer_size=10240)
    dataset = dataset.map(load_image_with_label, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
path_file = "playground/"
class_names = ["normal", "defect", "serius_defect"]


ds = tf_dataset_labels(path_file, 32, class_names)

plt.figure(figsize=(13, 13))
for images, labels in ds.take(1):
    for i in range(12):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.savefig('viewer_custom.png')